<table class="tfo-notebook-buttons" align="left">
  <td>
    <a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/nlp/c3_w4_02_modified_triplet_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>    
  </td>
  <td>
    <a href="https://github.com/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/nlp/c3_w4_02_modified_triplet_loss.ipynb" target="_parent"><img src="https://raw.githubusercontent.com/martin-fabbri/colab-notebooks/master/assets/github.svg" alt="View On Github"/></a>  </td>
</table>

#  Modified Triplet Loss : Ungraded Lecture Notebook
In this notebook you'll see how to calculate the full triplet loss, step by step, including the mean negative and the closest negative. You'll also calculate the matrix of similarity scores.

## Background
This is the original triplet loss function:

$\mathcal{L_\mathrm{Original}} = \max{(\mathrm{s}(A,N) -\mathrm{s}(A,P) +\alpha, 0)}$

It can be improved by including the mean negative and the closest negative, to create a new full loss function. The inputs are the Anchor $\mathrm{A}$, Positive $\mathrm{P}$ and Negative $\mathrm{N}$.

$\mathcal{L_\mathrm{1}} = \max{(mean\_neg -\mathrm{s}(A,P)  +\alpha, 0)}$

$\mathcal{L_\mathrm{2}} = \max{(closest\_neg -\mathrm{s}(A,P)  +\alpha, 0)}$

$\mathcal{L_\mathrm{Full}} = \mathcal{L_\mathrm{1}} + \mathcal{L_\mathrm{2}}$

Let me show you what that means exactly, and how to calculate each step.

## Imports

In [1]:
import numpy as np

## Similarity Scores
The first step is to calculate the matrix of similarity scores using cosine similarity so that you can look up $\mathrm{s}(A,P)$, $\mathrm{s}(A,N)$ as needed for the loss formulas.

### Two Vectors
First I'll show you how to calculate the similarity score, using cosine similarity, for 2 vectors.

$\mathrm{s}(v_1,v_2) = \mathrm{cosine \ similarity}(v_1,v_2) = \frac{v_1 \cdot v_2}{||v_1||~||v_2||}$
* Try changing the values in the second vector to see how it changes the cosine similarity.

In [2]:
def cosine_similarity(v1, v2):
    numerator = np.dot(v1, v2)
    denominator = np.sqrt(np.dot(v1, v1)) * np.sqrt(np.dot(v2, v2))
    return numerator / denominator

In [3]:
# two vector example
print("-- inputs --")
v1 = np.array([1, 2, 3], dtype=float)
v2 = np.array([1, 2, 3.5], dtype=float)
print("v1", v1)
print("v2", v2)
print("-- outputs --")
print("cosine similarity:", cosine_similarity(v1, v2))

-- inputs --
v1 [1. 2. 3.]
v2 [1.  2.  3.5]
-- outputs --
cosine similarity: 0.9974086507360697


In [4]:
print("-- inputs --")
v1 = np.array([1, 2, 3], dtype=float)
v2 = np.array([-1000, -20480, -7493.5], dtype=float)
print("-- outputs --")
print("cosine similarity:", cosine_similarity(v1, v2))

-- inputs --
-- outputs --
cosine similarity: -0.78890668575344


In [5]:
# two vector example
print("-- inputs --")
v1 = np.array([1, 2, 3], dtype=float)
v2 = np.array([1, 2, 3], dtype=float)
print("v1", v1)
print("v2", v2)
print("-- outputs --")
print("cosine similarity:", cosine_similarity(v1, v2))

-- inputs --
v1 [1. 2. 3.]
v2 [1. 2. 3.]
-- outputs --
cosine similarity: 1.0


### Two Batches of Vectors
Now i'll show you how to calculate the similarity scores, using cosine similarity, for 2 batches of vectors. These are rows of individual vectors, just like in the example above, but stacked vertically into a matrix. They would look like the image below for a batch size (row count) of 4 and embedding size (column count) of 5.

The data is setup so that $v_{1\_1}$ and $v_{2\_1}$ represent duplicate inputs, but they are not duplicates with any other rows in the batch. This means $v_{1\_1}$ and $v_{2\_1}$ (green and green) have more similar vectors than say $v_{1\_1}$ and $v_{2\_2}$ (green and magenta).

I'll show you two different methods for calculating the matrix of similarities from 2 batches of vectors.

<img src="https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/nlp/images/v1v2_stacked.png" width="450px"/>

### Inputs

In [6]:
v1_1 = np.array([1, 2, 3], dtype=np.float)
v1_2 = np.array([9, 2, 3], dtype=np.float)
v1_3 = np.array([-1, -4, -2], dtype=np.float)
v1_4 = np.array([1, -7, 2], dtype=np.float)
v1 = np.vstack([v1_1, v1_2, v1_3, v1_4])
print("v1:")
print(v1, "\n")
v2_1 = v1_1 + np.random.normal(0, 2, 3)
v2_2 = v1_2 + np.random.normal(0, 2, 3)
v2_3 = v1_3 + np.random.normal(0, 2, 3)
v2_4 = v1_4 + np.random.normal(0, 2, 3)
v2 = np.vstack([v2_1, v2_2, v2_3, v2_4])
print("v2:")
print(v2, "\n")

b = len(v1)
print("batch sizes match:", b == len(v2), "\n")
print("b ->", b)

v1:
[[ 1.  2.  3.]
 [ 9.  2.  3.]
 [-1. -4. -2.]
 [ 1. -7.  2.]] 

v2:
[[ 1.84056115  4.54193882  4.88315626]
 [ 8.0820375   0.38592027  2.57563085]
 [-0.68720154 -1.88816607 -2.92273731]
 [-1.1576644  -7.99054344  3.29250619]] 

batch sizes match: True 

b -> 4


### Outputs

In [7]:
# Similarity scores
sim_1 = np.zeros([b, b])
for row in range(sim_1.shape[0]):
    for col in range(sim_1.shape[1]):
        sim_1[row, col] = cosine_similarity(v1[row], v2[col])

print("option 1: loop")
print(sim_1, "\n")

option 1: loop
[[ 0.9879552   0.52187515 -0.99704802 -0.22256408]
 [ 0.60079736  0.98677072 -0.54465553 -0.19544377]
 [-0.93916281 -0.37975366  0.86660393  0.66407225]
 [-0.39707504  0.16878506  0.25646855  0.9576468 ]] 



Option 2: vector normalization and product

In [9]:
def norm(x):
    return x / np.sqrt(np.sum(x * x, axis=1, keepdims=True))

In [10]:
sim_2 = np.dot(norm(v1), norm(v2).T)
print("option 2 : vec norm & dot product")
print(sim_2, "\n")

option 2 : vec norm & dot product
[[ 0.9879552   0.52187515 -0.99704802 -0.22256408]
 [ 0.60079736  0.98677072 -0.54465553 -0.19544377]
 [-0.93916281 -0.37975366  0.86660393  0.66407225]
 [-0.39707504  0.16878506  0.25646855  0.9576468 ]] 



In [11]:
# Check
print("outputs are the same :", np.allclose(sim_1, sim_2))

outputs are the same : True


## Hard Negative Mining

I'll now show you how to calculate the mean negative $mean\_neg$ and the closest negative $close\_neg$ used in calculating $\mathcal{L_\mathrm{1}}$ and $\mathcal{L_\mathrm{2}}$.


$\mathcal{L_\mathrm{1}} = \max{(mean\_neg -\mathrm{s}(A,P)  +\alpha, 0)}$

$\mathcal{L_\mathrm{2}} = \max{(closest\_neg -\mathrm{s}(A,P)  +\alpha, 0)}$

You'll do this using the matrix of similarity scores you already know how to make, like the example below for a batch size of 4. The diagonal of the matrix contains all the $\mathrm{s}(A,P)$ values, similarities from duplicate question pairs (aka Positives). This is an important attribute for the calculations to follow.

<img src="https://github.com/martin-fabbri/colab-notebooks/raw/master/deeplearning.ai/nlp/images/ss_matrix.png" width="350px"/>


### Mean Negative
$mean\_neg$ is the average of the off diagonals, the $\mathrm{s}(A,N)$ values, for each row.

### Closest Negative
$closest\_neg$ is the largest off diagonal value, $\mathrm{s}(A,N)$, that is smaller than the diagonal $\mathrm{s}(A,P)$ for each row.
* Try using a different matrix of similarity scores. 